## Generative Deep Learning

In [22]:
# auto reloading
%reload_ext autoreload
%autoreload 2

In [23]:
# import
import numpy as np
import keras
from myutils import *

In [4]:
text = open('../../datasets/nietzsche.txt').read().lower()

In [5]:
print("Corpus length: %s" % str(len(text)))

Corpus length: 600893


In [8]:
max_len = 60  # seq of characters to train
step = 3      # 
sents = []
next_char = []

In [10]:
for i in range(0, len(text)-max_len, step):
    sents.append(text[i: i+max_len])
    next_char.append(text[i+max_len])

In [11]:
sents[:3]

['preface\n\n\nsupposing that truth is a woman--what then? is the',
 'face\n\n\nsupposing that truth is a woman--what then? is there ',
 'e\n\n\nsupposing that truth is a woman--what then? is there not']

In [12]:
next_char[:3]

['r', 'n', ' ']

In [13]:
text[:180]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand wo'

In [14]:
st = "this is me this calling atul"

In [21]:
word_index = word_index_generator(text)
word_index

{'retiring,': 0,
 'share': 1,
 'unfairly.': 2,
 '167.': 3,
 'kind': 4,
 'matures': 5,
 'possessing': 6,
 'george': 7,
 'fortify': 8,
 'turkish': 9,
 'foremost--old': 10,
 '150.': 11,
 'non-self-sufficient': 12,
 'observations")': 13,
 '"flesh': 14,
 'flung': 15,
 '(of': 16,
 '"out': 17,
 'four': 18,
 'adherence': 19,
 'dionysus': 20,
 'knowing,': 21,
 'martyred': 22,
 'sparks': 23,
 'selfless': 24,
 'gives': 25,
 'elevation:': 26,
 'woman"--she': 27,
 'motleyness.': 28,
 'window-pane': 29,
 '256.': 30,
 'roots': 31,
 '[2]': 32,
 'grasp,': 33,
 'entire,': 34,
 'plastic,': 35,
 'recognized--and': 36,
 'fanatics.': 37,
 'implicitness': 38,
 '"arms': 39,
 'conditionalness': 40,
 'laughing;': 41,
 'self-torture?': 42,
 'discouraging-encouraging': 43,
 'nonsense': 44,
 'experiments.': 45,
 'sermon': 46,
 '=survival': 47,
 'requirements': 48,
 '"get': 49,
 'approval': 50,
 'import': 51,
 'fairness,': 52,
 'soldiers,': 53,
 '=personal': 54,
 'brown': 55,
 'colder': 56,
 'found.': 57,
 'defence

In [25]:
char_index = char_index_generator(text)
char_index

{'\n': 29,
 ' ': 0,
 '!': 38,
 '"': 13,
 "'": 11,
 '(': 19,
 ')': 39,
 ',': 36,
 '-': 47,
 '.': 9,
 '0': 33,
 '1': 40,
 '2': 8,
 '3': 5,
 '4': 45,
 '5': 26,
 '6': 12,
 '7': 44,
 '8': 15,
 '9': 32,
 ':': 10,
 ';': 30,
 '=': 41,
 '?': 54,
 '[': 31,
 ']': 35,
 '_': 50,
 'a': 28,
 'b': 37,
 'c': 18,
 'd': 16,
 'e': 2,
 'f': 7,
 'g': 20,
 'h': 23,
 'i': 34,
 'j': 56,
 'k': 6,
 'l': 46,
 'm': 14,
 'n': 4,
 'o': 25,
 'p': 24,
 'q': 27,
 'r': 17,
 's': 55,
 't': 49,
 'u': 51,
 'v': 21,
 'w': 1,
 'x': 3,
 'y': 43,
 'z': 48,
 'ä': 52,
 'æ': 22,
 'é': 42,
 'ë': 53}

In [31]:
class mytest(object):
    def __init__(self, name, class_):
        self.name = name
        self.class_ = class_
        print("Hi %s"%self.name)

In [32]:
mine = mytest('atul', 12)

Hi atul


In [29]:
mine.name

'atul'

In [30]:
mine.class_

12

In [33]:
import json

In [35]:
strg = '{"name":"wala"}'

json.loads(strg)['name']

'wala'

In [36]:
from flask import request

In [37]:
abc = "[
('callback', 'jQuery111208315887275421632_1521463760575'), 
('[{\"doc_type\":\"text\",\"doc_name\":\"learnings.txt\"}]', ''), 
('_', '1521463760577')]"

In [40]:
mine.name[-2:]

'ul'